In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import utils

In [13]:
def read_in_data():
    train = pd.read_csv("input/train.csv")
    test = pd.read_csv("input/test.csv")
    breeds = pd.read_csv("input/breed_labels.csv")
    colours = pd.read_csv("input/color_labels.csv")
    return train, test, breeds, colours

def standardise_unnamed_pets(train, test):
    train["Name"] = train["Name"].fillna("Unnamed")
    train["Name"] = np.where(train["Name"] == "No Name", "Unnamed", train["Name"])
    train["Name"] = np.where(train["Name"] == "None", "Unnamed", train["Name"])
    train["Name"] = np.where(train["Name"] == "No Name Yet", "Unnamed", train["Name"])

    test["Name"] = test["Name"].fillna("Unnamed")
    test["Name"] = np.where(test["Name"] == "No Name", "Unnamed", test["Name"])
    test["Name"] = np.where(test["Name"] == "None", "Unnamed", test["Name"])
    test["Name"] = np.where(test["Name"] == "No Name Yet", "Unnamed", test["Name"])
    return train, test

def create_unnamed_feature(train, test):
    train["Unnamed"] = 0
    train.loc[train["Name"] == "Unnamed", "Unnamed"] = 1

    test["Unnamed"] = 0
    test.loc[test["Name"] == "Unnamed", "Unnamed"] = 1
    return train, test

def drop_na_values(train, test):
    train = train.fillna(0)
    test = test.fillna(0)
    return train, test

def create_mixed_breed_feature(train, test):
    train["MixedBreed"] = [1 if x > 0 else 0 for x in train["Breed2"]]
    test["MixedBreed"] = [1 if x > 0 else 0 for x in test["Breed2"]]
    return train, test

def breed_df_to_dict(breeds):
    breed_cols = breeds[["BreedID", "BreedName"]]
    breed_cols.set_index("BreedID", drop=True, inplace=True)
    breed_dict = breed_cols.to_dict()["BreedName"]
    return breed_dict

def map_dictionary_to_breed_names(breed_dict, train, test):
    train["BreedName_1"] = train["Breed1"].map(breed_dict).fillna("Unknown")
    train["BreedName_2"] = train["Breed2"].map(breed_dict).fillna("None")

    test["BreedName_1"] = test["Breed1"].map(breed_dict).fillna("Unknown")
    test["BreedName_2"] = test["Breed2"].map(breed_dict).fillna("None")

    # Make all the cases pure bred if breed 1 is the same as breed 2
    train.loc[train["BreedName_1"] == train["BreedName_2"], "MixedBreed"] = 0
    test.loc[test["BreedName_1"] == test["BreedName_2"], "MixedBreed"] = 0

    # But for the following cases, they are mixed:
    train.loc[train["BreedName_1"] == "Mixed Breed", "MixedBreed"] = 1
    train.loc[train["BreedName_1"] == "Domestic Short Hair", "MixedBreed"] = 1
    train.loc[train["BreedName_1"] == "Domestic Medium Hair", "MixedBreed"] = 1
    train.loc[train["BreedName_1"] == "Domestic Long Hair", "MixedBreed"] = 1

    test.loc[test["BreedName_1"] == "Mixed Breed", "MixedBreed"] = 1
    test.loc[test["BreedName_1"] == "Domestic Short Hair", "MixedBreed"] = 1
    test.loc[test["BreedName_1"] == "Domestic Medium Hair", "MixedBreed"] = 1
    test.loc[test["BreedName_1"] == "Domestic Long Hair", "MixedBreed"] = 1

    return train, test

def create_description_length_feature(train, test):
    train["Description"] = train["Description"].fillna("")
    train["Description_Character_Count"] = train["Description"].apply(lambda x: len(x))
    train["Description_Word_Count"] = train["Description"].apply(lambda x: len(x.split()))

    test["Description"] = test["Description"].fillna("")
    test["Description_Character_Count"] = test["Description"].apply(lambda x: len(x))
    test["Description_Word_Count"] = test["Description"].apply(lambda x: len(x.split()))
    return train, test

In [14]:
train, test, breeds, colours = read_in_data()
train, test = standardise_unnamed_pets(train, test)
train, test = create_unnamed_feature(train, test)
train, test = create_mixed_breed_feature(train, test)
breed_dict = breed_df_to_dict(breeds)
train, test = map_dictionary_to_breed_names(breed_dict, train, test)
train, test = create_description_length_feature(train, test)

train, test = drop_na_values(train, test)

In [15]:
def create_character_quantile_columns(df):
    character_25 = df["Description_Character_Count"].quantile(q=0.25)
    character_5 = df["Description_Character_Count"].quantile(q=0.5)
    character_75 = df["Description_Character_Count"].quantile(q=0.75)

    df["Character_25"] = 0
    df.loc[df["Description_Character_Count"] <= character_25, "Character_25"] = 1

    df["Character_5"] = 0
    df.loc[(df["Description_Character_Count"] > character_25) & (
            df["Description_Character_Count"] <= character_5), "Character_5"] = 1

    df["Character_75"] = 0
    df.loc[(df["Description_Character_Count"] > character_5) & (
            df["Description_Character_Count"] <= character_75), "Character_75"] = 1

    df["Character_100"] = 0
    df.loc[(df["Description_Character_Count"] > character_75), "Character_100"] = 1

    return df

def create_word_quantile_columns(df):
    word_25 = df["Description_Word_Count"].quantile(q=0.25)
    word_5 = df["Description_Word_Count"].quantile(q=0.5)
    word_75 = df["Description_Word_Count"].quantile(q=0.75)

    df["Word_25"] = 0
    df.loc[df["Description_Word_Count"] <= word_25, "Word_25"] = 1

    df["Word_5"] = 0
    df.loc[(df["Description_Word_Count"] > word_25) & (
            df["Description_Word_Count"] <= word_5), "Word_5"] = 1

    df["Word_75"] = 0
    df.loc[(df["Description_Word_Count"] > word_5) & (
            df["Description_Word_Count"] <= word_75), "Word_75"] = 1

    df["Word_100"] = 0
    df.loc[(df["Description_Word_Count"] > word_75), "Word_100"] = 1
    return df

def bucket_ages(df):
    # Drop Ages thats = 0
    df = df[df["Age"] > 0]
    df["Puppy"] = 0
    df["Adult"] = 0
    df["Senior"] = 0
    df.loc[(df["Age"] >= 1) & (df["Age"] < 12), "Puppy"] = 1
    df.loc[(df["Age"] >= 12) & (df["Age"] < 96), "Adult"] = 1
    df.loc[(df["Age"] >= 96, "Senior")] = 1
    return df

def bucket_fees(df):
    fee_25 = df["Fee"].quantile(q=0.25)
    fee_5 = df["Fee"].quantile(q=0.5)
    fee_75 = df["Fee"].quantile(q=0.75)
    df["No_Fee"] = 0
    df["Fee_25"] = 0
    df["Fee_5"] = 0
    df["Fee_75"] = 0
    df["Fee_100"] = 0
    df.loc[df["Fee"] == 0, "No_Fee"] = 1
    df.loc[(df["Fee"] > 0) & (
            df["Fee"] <= fee_25), "Fee_25"] = 1
    df.loc[(df["Fee"] > fee_25) & (
            df["Fee"] <= fee_5), "Fee_5"] = 1
    df.loc[(df["Fee"] > fee_5) & (
            df["Fee"] <= fee_75), "Fee_75"] = 1
    df.loc[(df["Fee"] > fee_75), "Fee_100"] = 1
    return df

In [16]:
train = create_character_quantile_columns(train)
test = create_character_quantile_columns(test)
train = create_word_quantile_columns(train)
test = create_word_quantile_columns(test)
train = bucket_ages(train)
test = bucket_ages(test)
train = bucket_fees(train)
test = bucket_fees(test)

Y_train = train["AdoptionSpeed"]

train = train.drop(
    columns=["AdoptionSpeed", "PetID", "Name", "State", "RescuerID", "Description", "BreedName_1", "BreedName_2",
            "Fee", "Description_Character_Count", "Description_Word_Count", "Age"])

test = test.drop(
    columns=["PetID", "Name", "State", "RescuerID", "Description", "BreedName_1", "BreedName_2",
            "Fee", "Description_Character_Count", "Description_Word_Count", "Age"])

data = train.append(test)
data = pd.get_dummies(data, columns = ["Type", "Gender", "MaturitySize", "FurLength", 
                                         "Vaccinated", "Dewormed", "Sterilized", 
                                         "Health", 'Breed1', 'Breed2', 'Color1', 'Color2', 'Color3', 'Quantity'])

In [23]:
X_train = data.iloc[:14814, :]
X_test = data.iloc[:14814, :]

In [26]:
#  Separate target variable from training set, and remove unnecessary columns
encoder = LabelEncoder()
encoder.fit(Y_train)
y_train = encoder.transform(Y_train)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)

batch_size = 256
epochs = 400
learning_rate = 0.0001

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(420,)))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

In [27]:
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=400,
                    verbose=1,
                    validation_split = 0.333)

Train on 9880 samples, validate on 4934 samples
Epoch 1/400
9880/9880 [==============================] - ETA: 0s - loss: 1.4600 - acc: 0.302 - 2s 156us/step - loss: 1.4585 - acc: 0.3033 - val_loss: 1.4120 - val_acc: 0.3308
Epoch 2/400
9880/9880 [==============================] - 1s 98us/step - loss: 1.4154 - acc: 0.3461 - val_loss: 1.4061 - val_acc: 0.3626
Epoch 3/400
9880/9880 [==============================] - 1s 99us/step - loss: 1.3863 - acc: 0.3778 - val_loss: 1.3926 - val_acc: 0.3620
Epoch 4/400
9880/9880 [==============================] - 1s 101us/step - loss: 1.3645 - acc: 0.3851 - val_loss: 1.3911 - val_acc: 0.3766
Epoch 5/400
9880/9880 [==============================] - 1s 98us/step - loss: 1.3430 - acc: 0.4053 - val_loss: 1.3738 - val_acc: 0.3861
Epoch 6/400
9880/9880 [==============================] - 1s 100us/step - loss: 1.3215 - acc: 0.4182 - val_loss: 1.4042 - val_acc: 0.3593
Epoch 7/400
9880/9880 [==============================] - 1s 97us/step - loss: 1.2991 - acc: 0.4

9880/9880 [==============================] - 1s 91us/step - loss: 0.1721 - acc: 0.9396 - val_loss: 4.3985 - val_acc: 0.3362
Epoch 61/400
9880/9880 [==============================] - 1s 89us/step - loss: 0.1382 - acc: 0.9488 - val_loss: 4.3716 - val_acc: 0.3328
Epoch 62/400
9880/9880 [==============================] - 1s 91us/step - loss: 0.1154 - acc: 0.9551 - val_loss: 4.9855 - val_acc: 0.3362
Epoch 63/400
9880/9880 [==============================] - 1s 92us/step - loss: 0.1553 - acc: 0.9463 - val_loss: 5.1007 - val_acc: 0.3356
Epoch 64/400
9880/9880 [==============================] - 1s 93us/step - loss: 0.1378 - acc: 0.9493 - val_loss: 4.5536 - val_acc: 0.3470
Epoch 65/400
9880/9880 [==============================] - 1s 93us/step - loss: 0.1187 - acc: 0.9538 - val_loss: 5.0247 - val_acc: 0.3456
Epoch 66/400
9880/9880 [==============================] - 1s 93us/step - loss: 0.1167 - acc: 0.9530 - val_loss: 4.8526 - val_acc: 0.3300
Epoch 67/400
9880/9880 [==============================

9880/9880 [==============================] - 1s 93us/step - loss: 0.0865 - acc: 0.9632 - val_loss: 6.2077 - val_acc: 0.3409
Epoch 120/400
9880/9880 [==============================] - 1s 91us/step - loss: 0.0628 - acc: 0.9700 - val_loss: 6.5918 - val_acc: 0.3245
Epoch 121/400
9880/9880 [==============================] - 1s 88us/step - loss: 0.0682 - acc: 0.9700 - val_loss: 6.8099 - val_acc: 0.3324
Epoch 122/400
9880/9880 [==============================] - 1s 90us/step - loss: 0.0657 - acc: 0.9699 - val_loss: 6.8592 - val_acc: 0.3452
Epoch 123/400
9880/9880 [==============================] - 1s 91us/step - loss: 0.0783 - acc: 0.9652 - val_loss: 6.5182 - val_acc: 0.3383
Epoch 124/400
9880/9880 [==============================] - 1s 93us/step - loss: 0.0874 - acc: 0.9636 - val_loss: 6.4030 - val_acc: 0.3383
Epoch 125/400
9880/9880 [==============================] - 1s 96us/step - loss: 0.0608 - acc: 0.9688 - val_loss: 6.3285 - val_acc: 0.3316
Epoch 126/400
9880/9880 [=======================

9880/9880 [==============================] - 1s 84us/step - loss: 0.0583 - acc: 0.9716 - val_loss: 7.0660 - val_acc: 0.3332
Epoch 179/400
9880/9880 [==============================] - 1s 87us/step - loss: 0.0517 - acc: 0.9738 - val_loss: 7.0508 - val_acc: 0.3387
Epoch 180/400
9880/9880 [==============================] - 1s 86us/step - loss: 0.0544 - acc: 0.9727 - val_loss: 7.4443 - val_acc: 0.3366
Epoch 181/400
9880/9880 [==============================] - 1s 85us/step - loss: 0.0567 - acc: 0.9707 - val_loss: 7.2042 - val_acc: 0.3433
Epoch 182/400
9880/9880 [==============================] - 1s 87us/step - loss: 0.0934 - acc: 0.9670 - val_loss: 7.1211 - val_acc: 0.3427
Epoch 183/400
9880/9880 [==============================] - 1s 83us/step - loss: 0.0571 - acc: 0.9718 - val_loss: 7.0556 - val_acc: 0.3340
Epoch 184/400
9880/9880 [==============================] - 1s 86us/step - loss: 0.0536 - acc: 0.9746 - val_loss: 6.8652 - val_acc: 0.3411
Epoch 185/400
9880/9880 [=======================

9880/9880 [==============================] - 1s 87us/step - loss: 0.0521 - acc: 0.9736 - val_loss: 6.9816 - val_acc: 0.3441
Epoch 238/400
9880/9880 [==============================] - 1s 87us/step - loss: 0.0472 - acc: 0.9745 - val_loss: 7.1854 - val_acc: 0.3368
Epoch 239/400
9880/9880 [==============================] - 1s 88us/step - loss: 0.0443 - acc: 0.9738 - val_loss: 7.3880 - val_acc: 0.3419
Epoch 240/400
9880/9880 [==============================] - 1s 88us/step - loss: 0.0575 - acc: 0.9720 - val_loss: 7.2090 - val_acc: 0.3364
Epoch 241/400
9880/9880 [==============================] - 1s 88us/step - loss: 0.0599 - acc: 0.9733 - val_loss: 7.4421 - val_acc: 0.3385
Epoch 242/400
9880/9880 [==============================] - 1s 95us/step - loss: 0.0479 - acc: 0.9748 - val_loss: 7.7917 - val_acc: 0.3370
Epoch 243/400
9880/9880 [==============================] - 1s 89us/step - loss: 0.0518 - acc: 0.9728 - val_loss: 7.3882 - val_acc: 0.3433
Epoch 244/400
9880/9880 [=======================

9880/9880 [==============================] - 1s 87us/step - loss: 0.0566 - acc: 0.9731 - val_loss: 7.0158 - val_acc: 0.3277
Epoch 297/400
9880/9880 [==============================] - 1s 87us/step - loss: 0.0437 - acc: 0.9766 - val_loss: 7.3330 - val_acc: 0.3368
Epoch 298/400
9880/9880 [==============================] - 1s 84us/step - loss: 0.0746 - acc: 0.9702 - val_loss: 6.9681 - val_acc: 0.3346
Epoch 299/400
9880/9880 [==============================] - 1s 84us/step - loss: 0.0582 - acc: 0.9742 - val_loss: 6.7658 - val_acc: 0.3281
Epoch 300/400
9880/9880 [==============================] - 1s 83us/step - loss: 0.0494 - acc: 0.9749 - val_loss: 7.4086 - val_acc: 0.3484
Epoch 301/400
9880/9880 [==============================] - 1s 85us/step - loss: 0.0428 - acc: 0.9761 - val_loss: 7.4774 - val_acc: 0.3328
Epoch 302/400
9880/9880 [==============================] - 1s 84us/step - loss: 0.0534 - acc: 0.9736 - val_loss: 7.1496 - val_acc: 0.3281
Epoch 303/400
9880/9880 [=======================

9880/9880 [==============================] - 1s 99us/step - loss: 0.0447 - acc: 0.9763 - val_loss: 7.8375 - val_acc: 0.3330
Epoch 356/400
9880/9880 [==============================] - 1s 94us/step - loss: 0.0539 - acc: 0.9741 - val_loss: 7.4520 - val_acc: 0.3381
Epoch 357/400
9880/9880 [==============================] - 1s 98us/step - loss: 0.0456 - acc: 0.9758 - val_loss: 7.6110 - val_acc: 0.3366
Epoch 358/400
9880/9880 [==============================] - 1s 96us/step - loss: 0.0573 - acc: 0.9739 - val_loss: 7.4950 - val_acc: 0.3275
Epoch 359/400
9880/9880 [==============================] - 1s 90us/step - loss: 0.0457 - acc: 0.9760 - val_loss: 7.7317 - val_acc: 0.3401
Epoch 360/400
9880/9880 [==============================] - 1s 92us/step - loss: 0.0764 - acc: 0.9716 - val_loss: 7.1488 - val_acc: 0.3346
Epoch 361/400
9880/9880 [==============================] - 1s 97us/step - loss: 0.0378 - acc: 0.9783 - val_loss: 7.6437 - val_acc: 0.3360
Epoch 362/400
9880/9880 [=======================

In [28]:
pred = (model.predict(X_test) > 0.5).astype("int32")

In [29]:
pred.shape

(14814, 5)

In [30]:
pred = pd.DataFrame(pred, columns=["Zero", "One", "Two", "Three", "Four"])

In [31]:
pred["AdoptionRate"] = 7
pred.loc[pred["Zero"] == 1, "AdoptionRate"] = 0
pred.loc[pred["One"] == 1, "AdoptionRate"] = 1
pred.loc[pred["Two"] == 1, "AdoptionRate"] = 2
pred.loc[pred["Three"] == 1, "AdoptionRate"] = 3
pred.loc[pred["Four"] == 1, "AdoptionRate"] = 4

pred.head()

,Zero,One,Two,Three,Four,AdoptionRate
0,0,0,1,0,0,2
1,1,0,0,0,0,0
2,0,0,0,1,0,3
3,0,0,1,0,0,2
4,0,0,1,0,0,2


In [32]:
submission_data = pd.read_csv("input/sample_submission.csv")

In [33]:
submission_data.shape

(3948, 2)

In [34]:
submission_data["AdoptionSpeed"] = pred["AdoptionRate"].astype("int32")
submission_data["AdoptionSpeed"] = submission_data["AdoptionSpeed"].fillna(0)
submission_data["AdoptionSpeed"] = submission_data["AdoptionSpeed"].astype("int32")
submission_data

,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,0
2,72000c4c5,3
3,e147a4b9f,2
4,43fbba852,2
5,77a490ec9,2
6,28c4b1b13,1
7,d1eada628,1
8,d134dec34,4
9,bcd464bb8,1
